<a href="https://colab.research.google.com/github/AnviTendulkar/Airline_Fare_Prediction/blob/master/fare_prediction1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [59]:
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler ,FunctionTransformer
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
import re
from datetime import datetime, timedelta
from sklearn.metrics import mean_squared_error
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import StandardScaler, PowerTransformer ,OneHotEncoder
from sklearn.compose import TransformedTargetRegressor
from sklearn.pipeline import FeatureUnion, Pipeline, make_pipeline
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.metrics import mean_absolute_error

In [42]:
pd.set_option("display.max_columns" ,None)
pd.set_option("display.max_rows" ,None)

In [43]:
train_data = pd.read_excel(r'/content/drive/MyDrive/Project/Airline Fare  Prediction/Data_Train.xlsx')

In [44]:
train_data.head(3)

,Airline,Date_of_Journey,Source,Destination,Route,Dep_Time,Arrival_Time,Duration,Total_Stops,Additional_Info,Price
0,IndiGo,24/03/2019,Banglore,New Delhi,BLR → DEL,22:20,01:10 22 Mar,2h 50m,non-stop,No info,3897
1,Air India,1/05/2019,Kolkata,Banglore,CCU → IXR → BBI → BLR,05:50,13:15,7h 25m,2 stops,No info,7662
2,Jet Airways,9/06/2019,Delhi,Cochin,DEL → LKO → BOM → COK,09:25,04:25 10 Jun,19h,2 stops,No info,13882


In [45]:
train_data.dropna(inplace=True)

In [46]:
X = train_data.loc[:,train_data.columns !='Price']
y =train_data['Price']

In [47]:
x_train ,x_test ,y_train ,y_test = train_test_split(X,y ,test_size=0.20)

In [48]:
print(x_train.head(1))
print(pd.DataFrame(x_test.head(1)))

        Airline Date_of_Journey   Source Destination      Route Dep_Time  \
4154  Air India       6/05/2019  Kolkata    Banglore  CCU → BLR    20:45   

     Arrival_Time Duration Total_Stops Additional_Info  
4154        23:20   2h 35m    non-stop         No info  
          Airline Date_of_Journey Source Destination            Route  \
4520  Jet Airways       9/05/2019  Delhi      Cochin  DEL → BOM → COK   

     Dep_Time  Arrival_Time Duration Total_Stops              Additional_Info  
4520    18:15  12:35 10 May  18h 20m      1 stop  In-flight meal not included  


In [49]:
# def hour_minute_extractor(Duration):
#     pattern = re.compile(r'(?:(\d+)h)?\s?(?:(\d+)m)?')
#     match = pattern.match(Duration)
#     if match:
#         hours = int(match.group(1)) if match.group(1) else 0
#         minutes = int(match.group(2)) if match.group(2) else 0
#         return pd.Series({'Hour' :hours , 'Minutes':minutes})

def Arrival_Date_calculation(row):
    pattern = re.compile(r'(?:(\d+)h)?\s?(?:(\d+)m)?')
    match = pattern.match(row['Duration'])
    if match:
        hours = int(match.group(1)) if match.group(1) else 0
        minutes = int(match.group(2)) if match.group(2) else 0
        # return pd.Series({'Hour' :hours , 'Minutes':minutes})

    # Concatenate 'Date_of_Journey' and 'Dep_Time'
    date_of_journey = row['Date_of_Journey'] + ' ' + row['Dep_Time']

    # Convert the concatenated string to datetime
    journey_date_time = datetime.strptime(date_of_journey, "%d/%m/%Y %H:%M")

    # Calculate arrival date by adding 'Duration_hour' and 'Duration_minute' to 'Date_of_Journey'
    arrival_date = journey_date_time + timedelta(hours=hours, minutes=minutes)
    return arrival_date

def split_dep_time_in_hours_min(str1):
    lst = str1.split(':')
    return pd.Series({'Dep_Hour': lst[0], 'Dep_Minute': lst[1]})

In [50]:
from sklearn.base import TransformerMixin
import pandas as pd

class DateColumnExtractor(TransformerMixin):
    def fit(self, X, y=None):
        return self

    def transform(self, X):
        temp_df =pd.DataFrame()
        X['Arrival_Date'] = X.apply(Arrival_Date_calculation ,axis=1)
        temp_df['Arrival_Year'] = X['Arrival_Date'].dt.year
        temp_df['Arrival_Month'] = X['Arrival_Date'].dt.month
        temp_df['Arrival_Day'] = X['Arrival_Date'].dt.day
        temp_df['Arrival_Hour'] = X['Arrival_Date'].dt.hour
        temp_df['Arrival_Minute'] = X['Arrival_Date'].dt.minute

        temp = X['Date_of_Journey'].copy()
        temp =  pd.to_datetime(temp)
        # Extract year, month, and day into separate columns
        temp_df['Dep_Year'] = temp.dt.year
        temp_df['Dep_Month'] = temp.dt.month
        temp_df['Dep_Day'] = temp.dt.day

        temp_df[['Dep_Hour', 'Dep_Minute']] = X['Dep_Time'].apply(split_dep_time_in_hours_min)
        return temp_df





In [51]:
x_train.head(1)

,Airline,Date_of_Journey,Source,Destination,Route,Dep_Time,Arrival_Time,Duration,Total_Stops,Additional_Info
4154,Air India,6/05/2019,Kolkata,Banglore,CCU → BLR,20:45,23:20,2h 35m,non-stop,No info


In [38]:
class replace_total_stops(TransformerMixin):
    def fit(self, X, y=None):
        return self
    def transform(self, X):
      return pd.DataFrame(X['Toatl_Stops'].replace({'1 stop': 1, 'non-stop': 0, '2 stops': 2, '3 stops': 3, '4 stops': 4}))


In [81]:
 df = x_train[['Airline','Date_of_Journey','Source','Destination','Dep_Time','Duration','Total_Stops']]

df.head(2)

,Airline,Date_of_Journey,Source,Destination,Dep_Time,Duration,Total_Stops
4154,Air India,6/05/2019,Kolkata,Banglore,20:45,2h 35m,non-stop
4208,GoAir,27/03/2019,Delhi,Cochin,07:00,5h 50m,1 stop


In [80]:
# # Display the transformed DataFrame

preprocessor = ColumnTransformer([
   ('one-hot',OneHotEncoder(drop = 'first' ,sparse_output =False ,dtype ='int16'),
     [0,2,3]),

   ('total_stop',FunctionTransformer(lambda x:x.replace({'1 stops':1,'1 stop':1, 'non-stop':0, '2 stops':2, '3 stops':3, '4 stops':4})),[df.columns.get_loc('Total_Stops')])
] ,remainder = 'passthrough')

pipe1 =Pipeline([('experimental_trans', DateColumnExtractor()),
                 ('preprocessor' ,preprocessor)])
clf = Pipeline(steps=[
                      ('pipe1', pipe1),
                      # ('linear_regression',LinearRegression()

                      # ('pipe1',pipe1),
                     ])



d1 =clf.fit_transform(df)
# data1 = pd.DataFrame(d1)
# print(data1.head(2))
print(pd.DataFrame(d1).head(3))


1


<ipython-input-50-d75e71d3e101>:46: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X['Arrival_Date'] = X.apply(Arrival_Date_calculation ,axis=1)
<ipython-input-50-d75e71d3e101>:56: UserWarning: Parsing dates in DD/MM/YYYY format when dayfirst=False (the default) was specified. This may lead to inconsistently parsed dates! Specify a format to ensure consistent parsing.
  temp =  pd.to_datetime(temp)


  0  1  2  3  4  5  6  7  8  9  10 11 12 13 14 15 16 17 18 19 20 21 22 23 24  \
0  0  0  0  0  1  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0   
1  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  1   
2  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  1  0  0  0   

  25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47 48 49  \
0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  1   
1  0  0  0  0  0  0  0  0  0  0  0  0  0  1  0  0  0  0  0  0  0  0  0  0  0   
2  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  1  0   

  50 51  52    53  54  55  56  
0  6  5  20  2019   5  20  45  
1  3  3  50  2019  27  07  00  
2  5  5  25  2019  24  21  05  


In [62]:
# score = clf.score(x_test, y_test)
y_pred = clf.predict(x_test)
mean_absolute_error(y_test,y_pred)

1


<ipython-input-50-d75e71d3e101>:56: UserWarning: Parsing dates in DD/MM/YYYY format when dayfirst=False (the default) was specified. This may lead to inconsistently parsed dates! Specify a format to ensure consistent parsing.
  temp =  pd.to_datetime(temp)


2642.374677757771

In [66]:
clf.get_params(deep=True)

{'memory': None,
 'steps': [('pipe1', Pipeline(steps=[('experimental_trans',
                    <__main__.DateColumnExtractor object at 0x7d7f36a56b00>),
                   ('preprocessor',
                    ColumnTransformer(remainder='passthrough',
                                      transformers=[('one-hot',
                                                     OneHotEncoder(drop='first',
                                                                   dtype='int16',
                                                                   sparse_output=False),
                                                     [0, 2, 3]),
                                                    ('total_stop',
                                                     FunctionTransformer(func=<function <lambda> at 0x7d7f369f1b40>),
                                                     [6])]))])),
  ('linear_regression', LinearRegression())],
 'verbose': False,
 'pipe1': Pipeline(steps=[('experimental_trans',
 

In [ ]:
# preprocessor = make_column_transformer(
#     (OneHotEncoder(drop = 'first' ,sparse_output =False ,dtype ='int16'),  ['Airline' ,'Source','Destination']),
#     (FunctionTransformer(lambda x:x.replace({'1 stops':1,'1 stop':1, 'non-stop':0, '2 stops':2, '3 stops':3, '4 stops':4})),['Total_Stops'])
# )

# pipe2 = make_pipeline(
#                        ('experimental_trans', DateColumnExtractor()),
#                        ('preprocessor', preprocessor)
# )
# df = x_train[['Airline','Date_of_Journey','Source','Destination','Dep_Time','Duration','Total_Stops']]
# data = pipe2.fit_transform(df,y_train)
# data1 = pd.DataFrame(data)
# print(data1.head(2))

In [77]:
pd.DataFrame(clf[:-1].transform(df),
             columns=clf[:-1].get_feature_names_out(),
             index=df.index)

IndexError: ignored

In [ ]:
x_train.head(1)

In [ ]:

from sklearn.impute import SimpleImputer
from sklearn.compose import make_column_transformer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.impute import SimpleImputer
from sklearn.linear_model import LinearRegression


SimpleImputer.get_feature_names_out = (lambda self, names=None:self.feature_names_in_)
print(SimpleImputer.get_feature_names_out)
num_pipeline = make_pipeline(SimpleImputer(), StandardScaler())
transformer = make_column_transformer(
    (num_pipeline, ["age", "height"]),
    (OneHotEncoder(), ["city"]))
pipeline = make_pipeline(transformer, LinearRegression())



df = pd.DataFrame({"city": ["Rabat", "Tokyo", "Paris", "Auckland"],
                   "age": [32, 65, 18, 24],
                   "height": [172, 163, 169, 190],
                   "weight": [65, 62, 54, 95]},
                  index=["Alice", "Bunji", "Cécile", "Dave"])



pipeline.fit(df, df["weight"])


## get pipeline feature names
print(pipeline[:-1].get_feature_names_out())


## specify feature names as your columns
pd.DataFrame(pipeline[:-1].transform(df),
             columns=pipeline[:-1].get_feature_names_out(),
             index=df.index)

In [ ]:
pipeline

In [ ]:
pipe2